In [2]:
import sc2reader
from sc2reader.engine.plugins import APMTracker, ContextLoader, SelectionTracker
from sc2reader import events
import pandas as pd
import glob


replays = []
paths = [path for path in glob.glob('./_data/**/*.SC2Replay', recursive=True)]

count = 10

for i, path in enumerate(paths[:count]):
    print('\r%6.2f%% complete' % ((i+1)/count*100), end='', flush=True)
    
    replay = sc2reader.load_replay(path, engine=sc2reader.engine.GameEngine(plugins=[ContextLoader(), APMTracker(), SelectionTracker()]))
    replays.append(replay)

100.00% complete

In [13]:
for event in replays[0].messages:
    print('%s\tmessage="%s"' % (event, event.text))

replays[0].players[0].name
print(replays[0].players[0], replays[0].players[0].result)

00.00	Seither         ChatEvent	message="glhf"
00.10	Probe           ChatEvent	message="gl hf"
14.28	Seither         ChatEvent	message="gg"
Player 1 - Seither (Terran) Loss


In [33]:
data = []

attrs_PlayerStatsEvent = ['frame','second','workers_active_count','minerals_collection_rate','vespene_collection_rate']
# attrs_UnitPositionsEvent = ['ability_type']

super_set = { 'P': set(), 'T': set() }

for i, replay in enumerate(replays[:10]):
	# print('\r%6.2f%% complete' % ((i+1)/count*100), end='', flush=True)

	if len(set([replay.players[0].pick_race[0],replay.players[1].pick_race[0]]) | set(['P','T'])) == 2:

		dd = {}

		for event in replay.events:
			if isinstance(event, events.PlayerLeaveEvent):
				print('Player', event.player, 'left', event.second, 'seconds into the game.')
				break

			if event.second % 30 == 0:

				# every 10 seconds
				if isinstance(event, events.PlayerStatsEvent):
					d = {}

					is_player_1 = replay.players[1].pid == event.pid
					name = replay.players[is_player_1].name
					race = replay.players[is_player_1].pick_race[0]
					win = replay.players[is_player_1].result == 'Win'

					lower_bound = 0 if event.second == 0 else event.second-30
					ap30s = sum(list(replay.players[is_player_1].aps.values())[lower_bound:event.second])

					d['match_id'] = i
					d['ap30s'] = ap30s
					d['race'] = race
					d['name'] = name
					d['win'] = win

					for attr in attrs_PlayerStatsEvent:
						d[attr] = eval('event.' + attr)
					
					dd[race] = d

				# every 15 seconds
				if isinstance(event, events.UnitPositionsEvent):
					# TODO: discriminate between races and map units

					data.extend(list(dd.values()))
					dd = {}

					s = set([str(a).split(' ')[0] for a in event.units.keys()])

					super_set['P'] = super_set['P'] | s
					super_set['T'] = super_set['T'] | s

					set_start = False

	# print(super_set)
pd.DataFrame(data)

Player Player 1 - Seither (Terran) left 869 seconds into the game.
Player Player 1 - Seither (Terran) left 1620 seconds into the game.
Player Player 1 - Probe (Protoss) left 978 seconds into the game.
Player Player 2 - Seither (Terran) left 1697 seconds into the game.


,match_id,ap30s,race,name,win,frame,second,workers_active_count,minerals_collection_rate,vespene_collection_rate
0,0,95.2,T,Seither,False,1920,120,18,671,179
1,0,156.8,P,Probe,True,1920,120,19,895,67
2,0,117.6,T,Seither,False,2880,180,19,755,156
3,0,124.6,P,Probe,True,2880,180,22,895,201
4,0,109.2,T,Seither,False,4320,270,22,1091,313
...,...,...,...,...,...,...,...,...,...,...
221,3,0.0,T,Seither,False,25920,1620,72,2071,492
222,3,0.0,P,Probe,True,26400,1650,75,3163,940
223,3,0.0,T,Seither,False,26400,1650,65,2239,403
224,3,0.0,P,Probe,True,26880,1680,75,3191,940


In [24]:
import pickle

with open('pieces.pickle', 'rb') as f:
    pieces = pickle.load(f)

pieces.keys()

dict_keys(['vespene_units', 'supply_units', 'worker_units', 'base_units', 'ground_units', 'air_units', 'tech_units', 'army_units', 'army_air', 'army_ground'])

In [34]:
super_set

{'Adept',
 'Archon',
 'Colossus',
 'Cyclone',
 'DarkTemplar',
 'Disruptor',
 'Ghost',
 'Hellion',
 'Immortal',
 'LiberatorAG',
 'MULE',
 'Marauder',
 'Marine',
 'Medivac',
 'Observer',
 'ObserverSiegeMode',
 'Oracle',
 'OrbitalCommand',
 'OrbitalCommandFlying',
 'Phoenix',
 'PlanetaryFortress',
 'Probe',
 'Raven',
 'Reaper',
 'SCV',
 'Sentry',
 'SiegeTank',
 'SiegeTankSieged',
 'Stalker',
 'Starport',
 'Viking',
 'WarpPrism',
 'WarpPrismPhasing',
 'WidowMine',
 'WidowMineBurrowed',
 'Zealot'}